In [ ]:
import pandas as pd
import random 
from tqdm import tqdm
import threading
import requests
import time

### Read the CSV file

### Load the file SUBRELOBJ.csv from https://zenodo.org/records/10022727

In [ ]:
SUBRELOBJ = pd.read_csv('RDF Files/SUBRELOBJ.csv')

In [ ]:
print("Number of rows: {}".format(SUBRELOBJ.shape))
print("Columns : {}".format(SUBRELOBJ.columns.tolist()))

In [ ]:
unique_entities = list(set(SUBRELOBJ['Subject'].tolist() + SUBRELOBJ['Object'].tolist()))
print("Number of unique entities : {}".format(len(unique_entities)))

## Query wikidata

In [ ]:
def get_entity_info(index):
    
    entity_name = unique_entities[index]
    # Construct the Wikidata API URL for the search query
    url = f"https://www.wikidata.org/w/api.php?action=wbsearchentities&search={entity_name}&language=en&format=json"

    # Send a GET request to the Wikidata API
    response = requests.get(url)

    # Check if the search was successful
    if response.status_code == 200:
        # Extract the first search result from the JSON response
        search_results = response.json().get("search")
        if search_results:
           
            entity_dict[entity_name]=search_results
        else:
            return None
    else:
        return None


In [ ]:

def arg_func(i):
    return i

def download_threader(func, max_num, threads, arg_func, N=None):
    
    for i in tqdm(range(0,max_num,threads)):
            
            limit=min(threads, max_num-i)
        
            parserThreads=[]
            
            for r in range(0,limit):
    
                parserThread=threading.Thread(target=func,args=(arg_func(i+r),))
                parserThreads.append(parserThread)    
                parserThread.start()
                    
            for parserThread in parserThreads:
                parserThread.join()
                
            
            if (i+1)%500==0:
                 print("Number of entities found : {}".format(len(entity_dict)))
                 time.sleep(100)

In [ ]:
entity_dict = {}
download_threader(get_entity_info, len(unique_entities), 9, arg_func)

In [ ]:
len(entity_dict)